# 보스턴 주택 가격 예측: 선형 회귀, 릿지, 라쏘 모델 비교

이 노트북은 보스턴 주택 가격 데이터셋을 사용하여 **선형 회귀(Linear Regression)**, **릿지 회귀(Ridge Regression)**, **라쏘 회귀(Lasso Regression)** 모델을 비교 분석합니다. 특히, 모델의 과대적합(Overfitting)을 방지하고 특성 선택(Feature Selection)을 수행하는 **규제(Regularization)**의 개념과 중요성을 중점적으로 다룹니다.

---
**주의**: 보스턴 주택 가격 데이터셋에는 인종(B)과 관련된 특성이 포함되어 있어 윤리적인 문제가 제기되었습니다. `scikit-learn`에서는 이 데이터셋을 더 이상 직접 제공하지 않으며, 본 노트북에서는 학습 목적으로 외부 URL에서 데이터를 로드하여 사용합니다.

### 1. 라이브러리 임포트


In [ ]:
import pandas as pd   
import numpy as np     
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression, Ridge, Lasso 
import matplotlib.pyplot as plt # 시각화를 위해 추가


### 2. 데이터 로드 및 준비

보스턴 주택 가격 데이터셋은 특정 형식으로 외부 URL에 저장되어 있습니다. `pandas.read_csv`와 `numpy.hstack`을 사용하여 데이터를 불러오고 특성(X)과 타겟(y)으로 분리합니다.


In [ ]:
url="http://lib.stat.cmu.edu/datasets/boston"
# 데이터 로드: 공백으로 구분되며, 처음 22줄은 건너뛰고 헤더가 없습니다.
df_raw = pd.read_csv(url, sep="\s+", skiprows=22, header=None)

# 데이터 결합: 홀수 행과 짝수 행을 결합하여 특성 행렬 X를 만듭니다.
# df.values[::2, :]는 짝수 인덱스 행의 모든 열을 가져옵니다.
# df.values[1::2, :2]는 홀수 인덱스 행의 처음 두 열을 가져옵니다.
X = np.hstack( [df_raw.values[::2, : ],  df_raw.values[1::2, :2]] )
# 타겟 변수 y는 홀수 인덱스 행의 세 번째 열입니다.
y = df_raw.values[1::2, 2]

print(f"특성 데이터 (X) 형태: {X.shape}")
print(f"타겟 데이터 (y) 형태: {y.shape}")
print("
특성 데이터 (X) 샘플 (상위 5개):
", X[:5])
print("
타겟 데이터 (y) 샘플 (상위 5개):
", y[:5])"


### 3. 데이터 분할

모델 학습과 평가를 위해 데이터를 훈련 세트와 테스트 세트로 분할합니다.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print(f"훈련 데이터 형태: {X_train.shape}, {y_train.shape}")
print(f"테스트 데이터 형태: {X_test.shape}, {y_test.shape}")


### 4. 선형 회귀 (Linear Regression)

가장 기본적인 회귀 모델입니다. 특성 간의 다중공선성(Multicollinearity) 문제를 고려하지 않으며, 특성 수가 많을 때 과대적합될 위험이 있습니다.


In [ ]:
model_lr = LinearRegression() 
model_lr.fit(X_train, y_train)

print("--- Linear Regression ---")
print(f"훈련 세트 R^2: {model_lr.score(X_train, y_train):.4f}")
print(f"테스트 세트 R^2: {model_lr.score(X_test, y_test):.4f}")
print("계수 (Coefficients):
", model_lr.coef_)
print(f"절편 (Intercept): {model_lr.intercept_:.4f}")


### 5. 릿지 회귀 (Ridge Regression)

릿지 회귀는 선형 회귀에 **L2 규제**를 추가한 모델입니다. 계수(가중치)의 크기를 제한하여 과대적합을 방지합니다. 계수를 완전히 0으로 만들지는 않습니다.

- **`alpha`**: 규제의 강도를 조절하는 하이퍼파라미터. `alpha`가 클수록 규제가 강해져 계수들이 0에 가까워집니다. `alpha=0`이면 일반 선형 회귀와 동일합니다.


In [ ]:
model_ridge = Ridge(alpha=10) 
model_ridge.fit(X_train, y_train)

print("--- Ridge Regression (alpha=10) ---")
print(f"훈련 세트 R^2: {model_ridge.score(X_train, y_train):.4f}")
print(f"테스트 세트 R^2: {model_ridge.score(X_test, y_test):.4f}")
print("계수 (Coefficients):
", model_ridge.coef_)
print(f"절편 (Intercept): {model_ridge.intercept_:.4f}")


### 6. 라쏘 회귀 (Lasso Regression)

라쏘 회귀는 선형 회귀에 **L1 규제**를 추가한 모델입니다. 릿지와 유사하게 과대적합을 방지하지만, 불필요한 특성의 계수를 완전히 0으로 만들어 **특성 선택(Feature Selection)** 효과를 가집니다.


In [ ]:
model_lasso = Lasso(alpha=10)  
model_lasso.fit(X_train, y_train)

print("--- Lasso Regression (alpha=10) ---")
print(f"훈련 세트 R^2: {model_lasso.score(X_train, y_train):.4f}")
print(f"테스트 세트 R^2: {model_lasso.score(X_test, y_test):.4f}")
print("계수 (Coefficients):
", model_lasso.coef_)
print(f"절편 (Intercept): {model_lasso.intercept_:.4f}")


### 7. 모델 비교 및 결론

세 가지 회귀 모델의 성능과 계수 변화를 비교하여 규제의 효과를 분석합니다.


In [ ]:
print("
--- 모델별 테스트 세트 R^2 비교 ---")
print(f"Linear Regression: {model_lr.score(X_test, y_test):.4f}")
print(f"Ridge Regression:    {model_ridge.score(X_test, y_test):.4f}")
print(f"Lasso Regression:    {model_lasso.score(X_test, y_test):.4f}")

print("
--- 계수 비교 (일부) ---")
print("Linear:  ", np.round(model_lr.coef_[:5], 2))
print("Ridge:   ", np.round(model_ridge.coef_[:5], 2))
print("Lasso:   ", np.round(model_lasso.coef_[:5], 2))

print("
--- 결론 ---")
print("1. **선형 회귀**: 가장 기본적인 모델이지만, 다중공선성이나 과대적합에 취약할 수 있습니다.")
print("2. **릿지 회귀**: 모든 특성의 계수를 0에 가깝게 줄여 과대적합을 방지합니다. 특성 수가 많고 모두 중요하다고 판단될 때 유용합니다.")
print("3. **라쏘 회귀**: 불필요한 특성의 계수를 완전히 0으로 만들어 특성 선택 효과를 가집니다. 중요한 특성만 남기고 싶을 때 유용합니다.")
print("
적절한 규제 강도(alpha)를 찾는 것이 중요하며, 이는 GridSearchCV나 Optuna와 같은 하이퍼파라미터 튜닝 기법을 통해 수행할 수 있습니다.")
